Bài tập phần tích Ecommerce Customers - Cao Nguyễn Huy Hoàng

In [ ]:
import pandas as pd 
import sklearn as sk 
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt 
import re
from time import time


from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error as MAE 
from sklearn.model_selection import train_test_split as TTP 

from sklearn.preprocessing import OneHotEncoder

# Import data
csv = pd.read_csv("../input/ecommerce-customers/Ecommerce Customers.csv")
ecom_df = pd.DataFrame(csv)
ecom_df.head(5)

In [ ]:
# DỮ liệu address không tồn tại, không dùng.

# 1. EDA



In [ ]:
ecom_df.describe()

In [ ]:
ecom_df.info()
# Có 500 dòng và có 500 non-null nên không có dữ liệu bị thiếu 

## Sơ bộ: 
* Data chỉ có biến phân loại là nhà cung cấp mail và thành phố, biến avatar có thể gây nhiễu và không liên quan
* Goal là số tiền spend và thời gian membership
=> Những trường nào có tương quan cao, vì thế ảnh hưởng đến goals? 

## Chưa rõ: 
* Thời gian ở đây đơn vị là gì? 
* Thời gian ở App và Web là tổng hay trung bình, liệu sự phân bố có quan trọng? 
* Khoảng tiền dùng nên nhìn theo số lượng hay phần trăm, liệu nhiều hơn có phải là nhiều hơn với người dùng? 
* Thời gian session được tính bằng cách nào? 

In [ ]:
# Biến email thành nhà phân phối email
def mailVendor(row):
    ven =re.findall('@\S*[^.com]', row.Email)[0][1:]
    if ven not in ['gmail', 'hotmail', 'yah']:
        ven = 'personal'
    row.Email = ven 
    return row
ecom_copy = ecom_df
ecom_df = ecom_copy.apply(mailVendor, axis='columns') 
ecom_df.Email

In [ ]:
ecom_df.Email.value_counts().plot.pie()
# Hơn một nữa số người dùng email riêng và nửa còn lại dùng 3 loại mail

In [ ]:
# Biến phân loại 
g = sns.FacetGrid(ecom_df, col="Email")
g.map(sns.histplot, "Yearly Amount Spent")




Không có thay sự khác biệt nhiều trong phân phối giữa các loại email, 
nhưng ở yah số lượng tiền dùng phân phối lệch trái hơn các hãng mail khác.

Hơn nữa trong dữ liệu, không biết mail của khách hàng là *** mail khi đăng ký hay đã đổi *** nên có thể bị thay đổi độc lập với các biến trên => có thể gây ra hiện tượng data leak. Vì thế, tốt nhất không sử dụng biến này.

In [ ]:
# Biến liên tục
g = sns.PairGrid(ecom_df)
g.map_diag(sns.histplot)
g.map_offdiag(sns.scatterplot, alpha = 0.3)

## Bình luận

### Phân phối: 
Các biến không có phân phối bình thường và không có dữ liệu ngoại lai. 
### Tương quan: 
Biến thời gian membership và biến số tiền đã trả có tương quan cao. Ngoài ra, biến Time on App và Avg. Session cũng có tương quan với số tiền bỏ ra 


In [ ]:
sns.pairplot(ecom_df, kind='reg', diag_kind='kde')

# 2. Linear regression

In [ ]:
# Tách bộ dữ liệu thành phần dùng để tạo model 
# và phần dùng để kiểm chứng độ chính xác 

# Feature là những biến đc chọn, loại biến phân loại
features = ['Avg. Session Length', 'Time on App', 'Time on Website', 'Length of Membership']
X = ecom_df[features]
# Y là nhãng hay biến ta muốn dự đoán, hay là biến phụ thuộc 
y = ecom_df['Yearly Amount Spent']
X_train, X_val, y_train, y_val = TTP(X, y, random_state=1)

In [ ]:
# Mô hình Linear Regression 
from sklearn.linear_model import LinearRegression as LR

# Tạo mô hình
lr = LR().fit(X_train, y_train)

# Dự đoán kết quả data mới là X_val, và so sánh với kết quả biết trước là y_Val
result = lr.predict(X_val)
mae = MAE(result, y_val)
print("Số tiền đoán sai trung bình là:" , mae)

In [ ]:
g = sns.scatterplot(x=result, y=y_val)
g.set(xlabel='Actual', ylabel='Predicted')

In [ ]:
coefs = pd.DataFrame(
    lr.coef_,
    columns=['Coefficients'],
    index=features
)
coefs

*** Hệ số tương quan có nghĩa là cứ mỗi 1 hệ số tăng, y sẽ thay đổi = 1 x coefficents ***

Kết luận: 
* Thời gian sử dụng tương quan nhiều nhất với tiền bỏ ra vì dĩ nhiên càng dùng nhiều càng trả nhiều
* Lượng thời gian sử dụng trên App và số thời gian sử dụng mỗi session cùng tăng cùng với số tiền 

=> Nên tập trung phát triển App và chú ý duy trì khách hàng dùng lâu?

Thời gian khách hàng sử dụng có thể thay đổi dựa trên nhiều điều kiện khác nhau, nếu vậy ép người dùng sử dụng lâu hơn họ sẽ trả nhiều tiền hơn? 

Vì không biết biến này thay đổi sau hay trước số tiêng mua và cũng không rõ cách thu thập và tính chất business model cùng với hệ số tương quan thấp, liệu biến có nên được dùng để inform decisions?

# 3. K-mean clustering 

In [ ]:
import numpy as np
from sklearn.datasets import load_digits


In [ ]:
# Gộp các bước xử lý và bình thường hóa dữ liệu 
# Train và trích thời gian để fit
# Đo lướng kết quả 
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
def bench_k_means(kmeans, name, data, labels):
    """Benchmark to evaluate the KMeans initialization methods.

    Parameters
    ----------
    kmeans : KMeans instance
        A :class:`~sklearn.cluster.KMeans` instance with the initialization
        already set.
    name : str
        Name given to the strategy. It will be used to show the results in a
        table.
    data : ndarray of shape (n_samples, n_features)
        The data to cluster.
    labels : ndarray of shape (n_samples,)
        The labels used to compute the clustering metrics which requires some
        supervision.
    """
    t0 = time()
    estimator = make_pipeline(StandardScaler(), kmeans).fit(data)
    fit_time = time() - t0
    results = [name, fit_time, estimator[-1].inertia_]

    # Define the metrics which require only the true labels and estimator
    # labels
    clustering_metrics = [
        metrics.homogeneity_score,
        metrics.completeness_score,
        metrics.v_measure_score,
        metrics.adjusted_rand_score,
        metrics.adjusted_mutual_info_score,
    ]
    results += [m(labels, estimator[-1].labels_) for m in clustering_metrics]

    # The silhouette score requires the full dataset
    results += [
        metrics.silhouette_score(data, estimator[-1].labels_,
                                 metric="euclidean", sample_size=300,)
    ]

    # Show the results
    formatter_result = ("{:9s}\t{:.3f}s\t{:.0f}\t{:.3f}\t{:.3f}"
                        "\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}")
    print(formatter_result.format(*results))

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

kmeans = KMeans(init="k-means++", n_clusters=3,random_state=0)
cluster_features = features + ["Yearly Amount Spent"]
kmeans.fit(ecom_df[cluster_features])

In [ ]:
clustered_df = ecom_df
clustered_df["cluster"] = kmeans.labels_

sns.pairplot(data=clustered_df,hue="cluster")

In [ ]:
# Trung bình
counts = clustered_df.cluster.value_counts()
agg = clustered_df.groupby('cluster').mean().reset_index()
agg

In [ ]:
clustered_df.groupby('cluster').sum().reset_index()

#### Bình luận:
* Số tiền spent tìm ra được 3 phân khúc. 
* Phần phối phân khúc này cũng có sự phân biệt ở biến membership

Từ kết luận về tương qua qua linear regression, nên tập trung những khách hàng tiềm năng hoặc đã là thành viên lâu năm vì nhóm khách hàng này chi nhiều nhất. 

Hơn 2 phân khúc khách hàng trả tiền nhiều nhất dùng app nhiều hơn, tập trung vào phát triển app sẽ lợi hơn so vs website - cả 3 nhóm dùng như nhau.

Tuy nhiên phân khúc khách hàng tầm trung không trả nhiều tiền nhất hằng năm nhưng đóng góp nhiều tiền nhất vào doanh số. 
Vì thế, tập trung duy trì nhóm khách hàng này thành thành viên lâu năm sẽ duy trì nhiều lợi doanh thu.

### Liệu có thể thấy được thông tin gì nếu chia nhiều cluster hơn? 

In [ ]:
kmeans = KMeans(init="k-means++", n_clusters=8,random_state=0)
kmeans.fit(ecom_df[cluster_features])
clustered_df["cluster2"] = kmeans.labels_

fig, axs = plt.subplots(ncols=2, figsize=(15,5))
sns.kdeplot(data=clustered_df,x="Yearly Amount Spent", hue="cluster", ax=axs[0]).set_title("3 cluster")
sns.kdeplot(data=clustered_df,x="Yearly Amount Spent", hue="cluster2", ax=axs[1]).set_title("8 cluster")


In [ ]:
cluster2 = clustered_df.groupby('cluster2').mean().reset_index()
cluster2

In [ ]:
sns.scatterplot(data = cluster2, y = "Length of Membership", x = "Time on App")

### Bình luận:
* Tail của nhóm dùng ít tiền nhất được xác định rõ hơn 
* Ở các nhóm thời gian tham gia có tương quan. 

=> Cải thiện app